In [5]:
import simpy
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import time
import csv
import os

# Kelas untuk menyimpan statistik simulasi
class ConcertTicketStats:
    def __init__(self):
        self.waiting_times = []
        self.service_times = []
        self.system_times = []
        self.queue_lengths = []
        self.time_stamps = []
        self.server_busy_times = defaultdict(float)
        self.server_utilization = defaultdict(float)
        self.abandonment_count = 0
        self.served_count = 0
        self.total_customers = 0

# Fungsi untuk menghasilkan waktu antar kedatangan (arrival) dengan distribusi eksponensial
def generate_interarrival_time(arrival_rate):
    return random.expovariate(arrival_rate)

# Fungsi untuk menghasilkan waktu layanan dengan distribusi normal
def generate_service_time(mean_service_time, std_service_time):
    service_time = random.normalvariate(mean_service_time, std_service_time)
    return max(0.5, service_time)  # Minimal 30 detik untuk menyelesaikan pemesanan

# Fungsi untuk menentukan apakah pelanggan akan meninggalkan antrian
def will_abandon(waiting_time, patience_time):
    return waiting_time > patience_time

# Proses pelanggan
def customer_process(env, customer_id, server, mean_service_time, std_service_time, 
                     stats, max_patience, ticket_types, sale_phase):
    # Waktu kedatangan pelanggan
    arrival_time = env.now
    
    # Mencatat jumlah total pelanggan
    stats.total_customers += 1
    
    # Mencatat panjang antrian saat pelanggan tiba
    current_queue_length = len(server.queue)
    stats.queue_lengths.append(current_queue_length)
    stats.time_stamps.append(arrival_time)
    
    # Tentukan waktu kesabaran pelanggan (berapa lama mereka mau menunggu)
    patience_time = random.expovariate(1.0/max_patience)
    
    # Jika pelanggan melihat antrian terlalu panjang, mereka mungkin langsung pergi
    if current_queue_length > 50 and random.random() < 0.3:
        stats.abandonment_count += 1
        return
    
    # Mulai proses permintaan server
    with server.request() as request:
        # Tunggu sampai server tersedia atau pelanggan menyerah
        results = yield request | env.timeout(patience_time)
        
        # Jika pelanggan menyerah
        if request not in results:
            stats.abandonment_count += 1
            return
        
        # Menghitung waktu tunggu
        waiting_time = env.now - arrival_time
        stats.waiting_times.append(waiting_time)
        
        # Waktu mulai layanan
        service_start = env.now
        
        # Pilih tipe tiket yang dibeli (berpengaruh pada waktu layanan)
        ticket_type = random.choice(ticket_types)
        
        # Layanan pemesanan tiket
        # Waktu layanan dipengaruhi oleh tipe tiket dan fase penjualan
        base_service_time = generate_service_time(mean_service_time, std_service_time)
        if ticket_type == "VIP":
            # VIP tiket membutuhkan waktu lebih lama untuk diproses
            service_time = base_service_time * 1.5
        elif ticket_type == "Regular":
            service_time = base_service_time
        else:  # Economy
            service_time = base_service_time * 0.8
            
        # Fase penjualan juga mempengaruhi waktu layanan
        if sale_phase == "peak":
            # Selama peak time, sistem sedikit lebih lambat karena beban tinggi
            service_time *= 1.2
        elif sale_phase == "presale":
            # Selama presale, mungkin ada verifikasi khusus
            service_time *= 1.1
            
        yield env.timeout(service_time)
        
        # Mencatat statistik layanan
        stats.service_times.append(service_time)
        stats.system_times.append(env.now - arrival_time)
        stats.served_count += 1
        
        # Mencatat waktu server sibuk
        server_id = id(request)
        stats.server_busy_times[server_id] += service_time

# Fungsi untuk menghasilkan pelanggan sesuai dengan fase penjualan
def customer_generator(env, server, stats, end_time, ticket_types, config):
    customer_id = 0
    current_phase_start = 0
    current_phase_index = 0
    
    # Dapatkan informasi fase penjualan
    phases = config["sale_phases"]
    
    # Generate pelanggan sampai waktu simulasi berakhir
    while env.now < end_time:
        # Periksa jika perlu berganti fase penjualan
        if current_phase_index < len(phases) - 1 and env.now >= phases[current_phase_index+1]["start_time"]:
            current_phase_index += 1
            current_phase_start = phases[current_phase_index]["start_time"]
            print(f"Beralih ke fase: {phases[current_phase_index]['name']} pada waktu {env.now:.1f}")
            
        # Gunakan parameter dari fase saat ini
        current_phase = phases[current_phase_index]
        arrival_rate = current_phase["arrival_rate"]
        mean_service_time = current_phase["mean_service_time"]
        std_service_time = current_phase["std_service_time"]
        max_patience = current_phase["max_patience"]
        
        # Buat pelanggan baru
        customer_id += 1
        env.process(customer_process(
            env, customer_id, server, 
            mean_service_time, std_service_time, stats, 
            max_patience, ticket_types, current_phase["name"]
        ))
        
        # Tunggu sampai pelanggan berikutnya tiba
        interarrival_time = generate_interarrival_time(arrival_rate)
        yield env.timeout(interarrival_time)

# Fungsi utama simulasi
def run_simulation(config, ticket_types, num_servers, simulation_time):
    # Buat lingkungan simulasi
    env = simpy.Environment()
    
    # Buat resource server dengan jumlah tertentu
    server = simpy.Resource(env, capacity=num_servers)
    
    # Inisialisasi statistik
    stats = ConcertTicketStats()
    
    # Mulai proses pembangkitan pelanggan
    env.process(customer_generator(env, server, stats, simulation_time, ticket_types, config))
    
    # Jalankan simulasi
    start_time = time.time()
    env.run(until=simulation_time)
    simulation_runtime = time.time() - start_time
    
    # Hitung utilisasi server
    for server_id, busy_time in stats.server_busy_times.items():
        stats.server_utilization[server_id] = busy_time / simulation_time
    
    # Hitung rata-rata utilisasi server
    avg_server_utilization = sum(stats.server_utilization.values()) / num_servers if num_servers > 0 else 0
    
    print(f"Simulasi selesai dalam {simulation_runtime:.2f} detik waktu nyata")
    return stats, avg_server_utilization

# Fungsi untuk menampilkan hasil simulasi
def display_results(stats, avg_utilization, num_servers, config):
    # Hitung metrik
    avg_waiting_time = np.mean(stats.waiting_times) if stats.waiting_times else 0
    avg_service_time = np.mean(stats.service_times) if stats.service_times else 0
    avg_system_time = np.mean(stats.system_times) if stats.system_times else 0
    avg_queue_length = np.mean(stats.queue_lengths) if stats.queue_lengths else 0
    abandonment_rate = (stats.abandonment_count / stats.total_customers * 100) if stats.total_customers > 0 else 0
    success_rate = (stats.served_count / stats.total_customers * 100) if stats.total_customers > 0 else 0
    
    print(f"\nHASIL SIMULASI SISTEM PEMESANAN TIKET KONSER")
    print(f"{'='*50}")
    print(f"Konfigurasi Simulasi:")
    print(f"- Jumlah Server: {num_servers}")
    print(f"- Durasi Simulasi: {config['simulation_time']} menit")
    print(f"- Fase Penjualan: {', '.join([phase['name'] for phase in config['sale_phases']])}")
    
    print(f"\nMetrik Kinerja:")
    print(f"- Total Pelanggan: {stats.total_customers}")
    print(f"- Pelanggan Dilayani: {stats.served_count} ({success_rate:.1f}%)")
    print(f"- Pelanggan Batal: {stats.abandonment_count} ({abandonment_rate:.1f}%)")
    print(f"- Rata-rata Waktu Tunggu: {avg_waiting_time:.2f} menit")
    print(f"- Rata-rata Waktu Layanan: {avg_service_time:.2f} menit")
    print(f"- Rata-rata Waktu dalam Sistem: {avg_system_time:.2f} menit")
    print(f"- Rata-rata Panjang Antrian: {avg_queue_length:.2f} pelanggan")
    print(f"- Utilisasi Server: {avg_utilization:.2%}")
    print(f"{'='*50}")
    
    return {
        'num_servers': num_servers,
        'avg_waiting_time': avg_waiting_time,
        'avg_service_time': avg_service_time,
        'avg_system_time': avg_system_time,
        'avg_queue_length': avg_queue_length,
        'server_utilization': avg_utilization,
        'abandonment_rate': abandonment_rate,
        'success_rate': success_rate,
        'total_customers': stats.total_customers,
        'served_customers': stats.served_count,
        'abandoned_customers': stats.abandonment_count
    }

# Fungsi untuk visualisasi hasil
def visualize_results(stats, config, scenario_name, result_dir):
    # Plot waktu tunggu
    plt.figure(figsize=(12, 6))
    plt.plot(stats.waiting_times)
    plt.axhline(y=np.mean(stats.waiting_times), color='r', linestyle='--', 
                label=f'Rata-rata: {np.mean(stats.waiting_times):.2f} menit')
    plt.title(f'Waktu Tunggu Pelanggan - {scenario_name}', fontsize=14)
    plt.xlabel('ID Pelanggan', fontsize=12)
    plt.ylabel('Waktu Tunggu (menit)', fontsize=12)
    plt.legend()
    plt.grid(True)
    
    # Simpan plot waktu tunggu
    waiting_time_file = os.path.join(result_dir, f'waktu_tunggu_{scenario_name.replace(" ", "_")}.png')
    plt.savefig(waiting_time_file, dpi=300, bbox_inches='tight')
    
    # Plot panjang antrian
    plt.figure(figsize=(12, 6))
    plt.plot(stats.time_stamps, stats.queue_lengths)
    
    # Tandai perubahan fase penjualan
    phases = config["sale_phases"]
    for i in range(1, len(phases)):
        plt.axvline(x=phases[i]["start_time"], color='g', linestyle='--', 
                   label=f'Mulai Fase {phases[i]["name"]}' if i == 1 else "")
    
    plt.title(f'Panjang Antrian dari Waktu ke Waktu - {scenario_name}', fontsize=14)
    plt.xlabel('Waktu Simulasi (menit)', fontsize=12)
    plt.ylabel('Panjang Antrian', fontsize=12)
    if len(phases) > 1:
        plt.legend()
    plt.grid(True)
    
    # Simpan plot panjang antrian
    queue_length_file = os.path.join(result_dir, f'panjang_antrian_{scenario_name.replace(" ", "_")}.png')
    plt.savefig(queue_length_file, dpi=300, bbox_inches='tight')
    
    plt.tight_layout()
    plt.close('all')  # Tutup semua plot untuk menghemat memori

# Fungsi untuk menyimpan hasil simulasi dalam format tabel CSV
def save_tabular_csv(results, result_dir):
    # Buat file hasil tabular CSV
    csv_file = os.path.join(result_dir, "hasil_simulasi_tabular.csv")
    
    with open(csv_file, 'w', newline='') as csvfile:
        # Tulis header
        csvfile.write("Metrik,")
        # Tambahkan kolom untuk setiap skenario jumlah server
        server_counts = [result['num_servers'] for result in results]
        csvfile.write(",".join([f"{server} Server" for server in server_counts]))
        csvfile.write("\n")
        
        # Definisikan metrik yang akan disimpan
        metrics = [
            ('Total Pelanggan', 'total_customers', '0f'),
            ('Pelanggan Dilayani', 'served_customers', '0f'),
            ('Pelanggan Batal', 'abandoned_customers', '0f'),
            ('Persentase Keberhasilan (%)', 'success_rate', '2f'),
            ('Persentase Pembatalan (%)', 'abandonment_rate', '2f'),
            ('Rata-rata Waktu Tunggu (menit)', 'avg_waiting_time', '2f'),
            ('Rata-rata Waktu Layanan (menit)', 'avg_service_time', '2f'),
            ('Rata-rata Waktu dalam Sistem (menit)', 'avg_system_time', '2f'),
            ('Rata-rata Panjang Antrian', 'avg_queue_length', '2f'),
            ('Utilisasi Server (%)', 'server_utilization', '2%')
        ]
        
        # Tulis setiap baris metrik
        for label, key, format_str in metrics:
            row = [label]
            for result in results:
                # Format nilai sesuai dengan tipe data
                if format_str.endswith('%'):
                    value = result[key]
                    formatted_value = f"{value:.{format_str[:-1]}%}"
                else:
                    value = result[key]
                    formatted_value = f"{value:.{format_str}}"
                row.append(formatted_value)
            csvfile.write(",".join(row))
            csvfile.write("\n")
    
    print(f"\nHasil simulasi dalam format tabel telah disimpan ke {csv_file}")

# Fungsi untuk menyimpan hasil ke CSV
def save_to_csv(results, result_dir):
    csv_file = os.path.join(result_dir, "hasil_simulasi.csv")
    
    with open(csv_file, 'w', newline='') as csvfile:
        # Tentukan kolom-kolom header
        fieldnames = [
            'num_servers', 
            'total_customers', 
            'served_customers', 
            'abandoned_customers',
            'success_rate', 
            'abandonment_rate', 
            'avg_waiting_time', 
            'avg_service_time', 
            'avg_system_time', 
            'avg_queue_length', 
            'server_utilization'
        ]
        
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        # Tulis setiap baris hasil
        for result in results:
            # Copy dan format data untuk CSV
            row = result.copy()
            # Konversi utilization ke persentase untuk CSV
            row['server_utilization'] = row['server_utilization'] * 100
            writer.writerow(row)
    
    print(f"\nHasil simulasi telah disimpan ke {csv_file}")

# Fungsi untuk membandingkan skenario
def compare_scenarios(results, server_counts, result_dir):
    metrics = ['avg_waiting_time', 'server_utilization', 'abandonment_rate']
    titles = ['Rata-rata Waktu Tunggu', 'Utilisasi Server', 'Tingkat Pembatalan']
    y_labels = ['Waktu (menit)', 'Utilisasi (%)', 'Pembatalan (%)']
    
    plt.figure(figsize=(18, 6))
    
    for i, metric in enumerate(metrics):
        plt.subplot(1, 3, i+1)
        values = [r[metric] for r in results]
        
        if metric == 'server_utilization':
            values = [v * 100 for v in values]  # Convert to percentage
            
        plt.bar(server_counts, values, color='skyblue')
        plt.title(titles[i], fontsize=14)
        plt.xlabel('Jumlah Server', fontsize=12)
        plt.ylabel(y_labels[i], fontsize=12)
        plt.grid(True, axis='y', linestyle='--', alpha=0.7)
        
        # Tambahkan nilai di atas bar
        for j, v in enumerate(values):
            plt.text(server_counts[j], v + max(values)*0.03, 
                    f"{v:.1f}" if metric != 'server_utilization' else f"{v:.1f}%", 
                    ha='center')
    
    plt.tight_layout()
    
    # Simpan grafik perbandingan
    comparison_file = os.path.join(result_dir, "perbandingan_skenario.png")
    plt.savefig(comparison_file, dpi=300, bbox_inches='tight')
    plt.close()

# Fungsi untuk melakukan analisis berbagai skenario
def analyze_scenarios():
    # Buat direktori Result jika belum ada
    result_dir = "Result"
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
        print(f"Direktori {result_dir} telah dibuat")
    
    # Parameter umum simulasi
    ticket_types = ["VIP", "Regular", "Economy"]
    
    # Definisikan fase penjualan tiket konser
    base_config = {
        "simulation_time": 480,  # 8 jam simulasi
        "sale_phases": [
            {
                "name": "presale",
                "start_time": 0,
                "arrival_rate": 1.0,  # pelanggan/menit
                "mean_service_time": 3.0,  # menit
                "std_service_time": 0.5,
                "max_patience": 20  # rata-rata kesabaran pelanggan dalam menit
            },
            {
                "name": "peak",
                "start_time": 60,  # dimulai setelah 1 jam
                "arrival_rate": 4.0,  # lonjakan permintaan
                "mean_service_time": 2.5,
                "std_service_time": 0.8,
                "max_patience": 15
            },
            {
                "name": "normal",
                "start_time": 180,  # dimulai setelah 3 jam
                "arrival_rate": 2.0,
                "mean_service_time": 2.0,
                "std_service_time": 0.4,
                "max_patience": 10
            }
        ]
    }
    
    # Uji berbagai jumlah server
    server_counts = [2, 4, 6, 8]
    results = []
    
    for num_servers in server_counts:
        print(f"\n{'#'*70}")
        print(f"SKENARIO: {num_servers} SERVER")
        print(f"{'#'*70}")
        
        # Jalankan simulasi
        stats, avg_utilization = run_simulation(
            base_config, ticket_types, num_servers, base_config["simulation_time"]
        )
        
        # Tampilkan hasil
        scenario_name = f"{num_servers} Server"
        metrics = display_results(stats, avg_utilization, num_servers, base_config)
        visualize_results(stats, base_config, scenario_name, result_dir)
        
        # Simpan hasil
        results.append(metrics)
    
    # Visualisasi perbandingan antar skenario
    compare_scenarios(results, server_counts, result_dir)
    
    # Simpan hasil dalam format CSV standar
    save_to_csv(results, result_dir)
    
    # Simpan hasil dalam format tabel CSV
    save_tabular_csv(results, result_dir)
    
    return results

# Jalankan analisis jika file ini dijalankan langsung
if __name__ == "__main__":
    results = analyze_scenarios()


######################################################################
SKENARIO: 2 SERVER
######################################################################
Beralih ke fase: peak pada waktu 61.4
Beralih ke fase: normal pada waktu 180.4
Simulasi selesai dalam 0.02 detik waktu nyata

HASIL SIMULASI SISTEM PEMESANAN TIKET KONSER
Konfigurasi Simulasi:
- Jumlah Server: 2
- Durasi Simulasi: 480 menit
- Fase Penjualan: presale, peak, normal

Metrik Kinerja:
- Total Pelanggan: 1171
- Pelanggan Dilayani: 366 (31.3%)
- Pelanggan Batal: 788 (67.3%)
- Rata-rata Waktu Tunggu: 11.31 menit
- Rata-rata Waktu Layanan: 2.61 menit
- Rata-rata Waktu dalam Sistem: 13.94 menit
- Rata-rata Panjang Antrian: 26.26 pelanggan
- Utilisasi Server: 99.69%

######################################################################
SKENARIO: 4 SERVER
######################################################################
Beralih ke fase: peak pada waktu 60.4
Beralih ke fase: normal pada waktu 180.5
Simulasi selesai d